In [ ]:
#required packages

from datetime import datetime
from meteostat import Stations, Hourly
import pandas as pd
from prophet import Prophet
import numpy as np

In [ ]:
##This entire cell focuses on finding the nearest weather station with given coordinates. The script then proceeds to make a csv file of data from 
##the stationat a given start and end time. Missing data in the chart is filled by normalizing per meteostat documentation

stations = Stations()
stations = stations.nearby(31.3573, -100.5028)
station = stations.fetch(1)

start = datetime(2018, 1, 1)
end = datetime(2023, 3, 7, 23, 59)

data = Hourly(station, start=start, end=end)
data = data.normalize()
data = data.interpolate()
data = data.fetch()
dataset=data
data.to_csv('base_data')

In [ ]:
##This Cell allows you to review the data, place it into a table, and get basic statistics

dataset='base_data'
table=pd.read_csv(dataset)
print(table.head())
print(table.info())
print(table.describe(exclude=[object]))
table.dtypes

In [ ]:
##This cell makes the data ready for Neural Phrophet learning and forecasting

table['ds']=pd.DatetimeIndex(table['temp']) #Refer to Meteostat guide for variable abbreviations, generally, the colum titles are avaliable variables
data=table[['time','temp']]
data.columns=['ds','y']
data.head
data.head
df=data

In [ ]:
##This cell is the machine learning part, frequency is 'H' as this model uses hourly data. epochs is the number of times the machine runs the model
##The more epochs, the more accurate the model becomes, but it will be slower and there is will be a point where its unpractical to have a high epoch

from neuralprophet import NeuralProphet
m = NeuralProphet()
metrics = m.fit(df, freq='H', epochs=25)
forecast = m.predict(df)
m.plot(forecast) #This cell shows you the machines general prediction results. If bluelines are not in a reasonable line, you may need more epochs

In [ ]:
##This cell focuses on getting a forecast, yhat is what the machine predicts, but other useful data comes out like daily, weekly, and seasonal
##daily differences. periods tells the machine how many hours to forecast for from the end of the dataset. The forecast is saved in csv.

future=m.make_future_dataframe(data, periods=120)
forecast=m.predict(future)
forecast.head()
forecast.to_csv('forecast_data')
forecast